# Convert OOF and Submission Predictions into a Video!
This notebook converts your `submission.csv` or `oof.csv` file into a video that you can watch to analyze your model. The video displays ground truths and model predictions of all three video views simultaneously. After creating a video with this notebook, it is best to watch, by scrolling to this notebook's Output Section, and then watching the video in the Output Section. Then you will be able to click the full screen button. If you watch it embedded in the Jupyter Notebook, the full screen button doesn't work. The boxes in the video follow the following key:

**Key:**  
Red - ground truth impact  
White - warns you that ground truth is coming within 10 frames  
Black - warns you that ground truth occurred within 10 frames in past  
  
Blue - model prediction   
Yellow - warns you that prediction is coming within 10 frames  
Green - warns you that prediction occurred within 10 frames in past  

In [ ]:
import numpy as np, pandas as pd, os
pd.options.display.max_rows = 999

OOF_FILE = '../input/nfl-oof-0513/OOF_CV_513.csv'
df_pred = pd.read_csv(OOF_FILE)
df_pred.head()

# Load Train Labels

In [ ]:
import cv2
from tqdm.notebook import tqdm
import subprocess
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from IPython.display import Video, display

import datetime as dt
from pathlib import Path

In [ ]:
BASE_DIR = '../input/nfl-impact-detection/'

# LOAD TRAIN LABELS
train = pd.read_csv(BASE_DIR+'train_labels.csv')
print('Train label shape is', train.shape )
train.head()

In [ ]:
# FIND TARGETS AND LABEL THEIR COMING AND GOING
train['target'] = ((train.impact==1)&(train.confidence>1)&(train.visibility>0)).astype('int8')
train['warning'] = 0
train = train.sort_values(['video','label','frame']).reset_index(drop=True)

# MARK PREVIOUS AND FOLLOWING IMPACT SO WE CAN WARN VIEWERS
for k in range(-10,1):
    train.warning += train.target.shift(k).fillna(0)
for k in range(1,11):
    train.warning -= train.target.shift(k).fillna(0)
    
train['hit'] = train.groupby(['video','frame']).target.transform('max')

# Load Tracking Data

In [ ]:
# LOAD TRACKING DATA
track = pd.read_csv(BASE_DIR+'train_player_tracking.csv')
track["time"] = pd.to_datetime(track["time"])
track["color"] = track["player"].map(lambda x: "black" if "H" in x else "white")
print( track.shape )
track.head()

In [ ]:
# https://www.kaggle.com/robikscube/nfl-big-data-bowl-plotting-player-position/notebook
def create_football_field(linenumbers=True,
                          endzones=True,
                          highlight_line=False,
                          highlight_line_number=50,
                          highlighted_name='Line of Scrimmage',
                          fifty_is_los=False,
                          figsize=(12, 6.33)):
    """
    Function that plots the football field for viewing plays.
    Allows for showing or hiding endzones.
    """
    rect = patches.Rectangle((0, 0), 120, 53.3, linewidth=0.1,
                             edgecolor='r', facecolor='forestgreen', zorder=0)  # changed the field color to forestgreen

    fig, ax = plt.subplots(1, figsize=figsize)
    ax.add_patch(rect)

    plt.plot([10, 10, 10, 20, 20, 30, 30, 40, 40, 50, 50, 60, 60, 70, 70, 80,
              80, 90, 90, 100, 100, 110, 110, 120, 0, 0, 120, 120],
             [0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3,
              53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 53.3, 0, 0, 53.3],
             color='white')
    if fifty_is_los:
        plt.plot([60, 60], [0, 53.3], color='gold')
        plt.text(62, 50, '<- Player Yardline at Snap', color='gold')
    # Endzones
    if endzones:
        ez1 = patches.Rectangle((0, 0), 10, 53.3,
                                linewidth=0.1,
                                edgecolor='r',
                                facecolor='blue',
                                alpha=0.2,
                                zorder=0)
        ez2 = patches.Rectangle((110, 0), 120, 53.3,
                                linewidth=0.1,
                                edgecolor='r',
                                facecolor='blue',
                                alpha=0.2,
                                zorder=0)
        ax.add_patch(ez1)
        ax.add_patch(ez2)
    plt.xlim(0, 120)
    plt.ylim(-5, 58.3)
    plt.axis('off')
    if linenumbers:
        for x in range(20, 110, 10):
            numb = x
            if x > 50:
                numb = 120 - x
            plt.text(x, 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white')
            plt.text(x - 0.95, 53.3 - 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white', rotation=180)
    if endzones:
        hash_range = range(11, 110)
    else:
        hash_range = range(1, 120)

    for x in hash_range:
        ax.plot([x, x], [0.4, 0.7], color='white')
        ax.plot([x, x], [53.0, 52.5], color='white')
        ax.plot([x, x], [22.91, 23.57], color='white')
        ax.plot([x, x], [29.73, 30.39], color='white')

    if highlight_line:
        hl = highlight_line_number + 10
        plt.plot([hl, hl], [0, 53.3], color='yellow')
        plt.text(hl + 2, 50, '<- {}'.format(highlighted_name),
                 color='yellow')
    return fig, ax

#create_football_field()
#plt.show()

In [ ]:
def get_track_image(gameKey=58000, playID=1306, fps=60, frame=10, fmax=300, labels=False, 
                     warn1=[], warn2=[], hit=[]):

    play_track = track.loc[(track.gameKey == gameKey) & (track.playID == playID)]
    snap_time = play_track.loc[play_track.event == 'ball_snap','time'].iloc[0]

    play_start = snap_time - dt.timedelta(seconds = 0.1) #10/fps)
    play_end = play_start + dt.timedelta(seconds = fmax/fps)
    play_track = play_track.loc[(play_track.time>=play_start)&(play_track.time<=play_end)]

    current = play_start + dt.timedelta(seconds = frame/fps)
    abs_timedelta = abs(play_track['time'] - current).dt.total_seconds()
    min_abs_timedelta = abs_timedelta.min()
    play_current = play_track[abs_timedelta == min_abs_timedelta]

    fig, ax = create_football_field(figsize=(20, 12))
    play_current.plot(x="x", y="y",  kind='scatter', ax=ax, color = play_current['color'], s=100)
    
    draw = [warn1,warn2,hit]
    colors = ['white','black','blue']
    widths = [2,2,5]
    
    for i,j in enumerate(draw):
        for k in j:
            row = play_current.loc[(play_current.player==k)]
            if len(row)==0: continue
            row = row.iloc[0]
            ax.plot([row.x-1,row.x-1],[row.y-1,row.y+1],color=colors[i],linewidth=widths[i])
            ax.plot([row.x+1,row.x+1],[row.y-1,row.y+1],color=colors[i],linewidth=widths[i])
            ax.plot([row.x-1,row.x+1],[row.y-1,row.y-1],color=colors[i],linewidth=widths[i])
            ax.plot([row.x-1,row.x+1],[row.y+1,row.y+1],color=colors[i],linewidth=widths[i])

    if labels:
        play_home = play_current.loc[play_current['player'].str.contains('H')]
        play_away = play_current.loc[play_current['player'].str.contains('V')]
        for index, row in play_away.iterrows():
            ax.annotate(row['player'], (row['x'], row['y']), verticalalignment='center', horizontalalignment='right', fontsize=12)
        for index, row in play_home.iterrows():
            ax.annotate(row['player'], (row['x'], row['y']), verticalalignment='center', horizontalalignment='left', 
                    color = 'white', fontsize=12)
    
    #Image from plot
    ax.axis('off')
    fig.tight_layout(pad=0)

    # To remove the huge white borders
    ax.margins(0)

    fig.canvas.draw()
    image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    image_from_plot = image_from_plot.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.close()

    return image_from_plot[72:-72,80:-80,:]

In [ ]:
# DISPLAY GAME, PLAY, FRAME, AND MARK HELMET LABELS
# VARIABLE WARN1, WARN2, AND HIT ARE LISTS OF HELMET LABELS
img = get_track_image(57787,3413,frame=50,warn1=['H68'])
plt.figure(figsize=(15,7))
plt.imshow(img[::2,::2,::-1])
plt.show()

In [ ]:
img.shape

# OOF Predictions

In [ ]:
valid_videos = df_pred.video.unique()
print('There are', len(valid_videos),'unique videos in OOF file.')
print('OOF file shape is', df_pred.shape )
df_pred.head()

In [ ]:
# https://www.kaggle.com/nvnnghia/evaluation-metrics
from scipy.optimize import linear_sum_assignment

def iou_ltwh(bbox1, bbox2):
    
    
    bbox1 = [float(x) for x in bbox1]
    bbox2 = [float(x) for x in bbox2]
    
    bbox1[2] += bbox1[0] 
    bbox1[3] += bbox1[1] 
    
    bbox2[2] += bbox2[0] 
    bbox2[3] += bbox2[1] 

    (x0_1, y0_1, x1_1, y1_1) = bbox1
    (x0_2, y0_2, x1_2, y1_2) = bbox2

    # get the overlap rectangle
    overlap_x0 = max(x0_1, x0_2)
    overlap_y0 = max(y0_1, y0_2)
    overlap_x1 = min(x1_1, x1_2)
    overlap_y1 = min(y1_1, y1_2)

    # check if there is an overlap
    if overlap_x1 - overlap_x0 <= 0 or overlap_y1 - overlap_y0 <= 0:
            return 0

    # if yes, calculate the ratio of the overlap to each ROI size and the unified size
    size_1 = (x1_1 - x0_1) * (y1_1 - y0_1)
    size_2 = (x1_2 - x0_2) * (y1_2 - y0_2)
    size_intersection = (overlap_x1 - overlap_x0) * (overlap_y1 - overlap_y0)
    size_union = size_1 + size_2 - size_intersection

    return size_intersection / size_union

def precision_calc(gt_boxes, pred_boxes):
    cost_matix = np.ones((len(gt_boxes), len(pred_boxes)))
    for i, box1 in enumerate(gt_boxes):
        for j, box2 in enumerate(pred_boxes):
            dist = abs(box1[0]-box2[0])
            if dist > 4:
                continue
            iou_score2 = iou_ltwh(box1[1:], box2[1:])

            if iou_score2 < 0.35:
                continue
            else:
                cost_matix[i,j]=0

    row_ind, col_ind = linear_sum_assignment(cost_matix)
    fn = len(gt_boxes) - row_ind.shape[0]
    fp = len(pred_boxes) - col_ind.shape[0]
    tp=0
    for i, j in zip(row_ind, col_ind):
        if cost_matix[i,j]==0:
            tp+=1
        else:
            fp+=1
            fn+=1
    return tp, fp, fn

def competition_metric(valid_labels, pred_df, output=False):
    ftp, ffp, ffn = [], [], []
    cols = ['frame', 'left', 'top', 'width', 'height']
    for video in valid_labels['video'].unique():
        pred_boxes = pred_df[pred_df['video'] == video][cols].values
        gt_boxes = valid_labels[valid_labels['video'] == video][cols].values
       
        tp, fp, fn = precision_calc(gt_boxes, pred_boxes)
        ftp.append(tp)
        ffp.append(fp)
        ffn.append(fn)
    
    tp = np.sum(ftp)
    fp = np.sum(ffp)
    fn = np.sum(ffn)
    precision = tp / (tp + fp + 1e-6)
    recall =  tp / (tp + fn +1e-6)
    f1_score = 2*(precision*recall)/(precision+recall+1e-6)
    if output:
        return tp, fp, fn, precision, recall, f1_score
    else:
        print(f'TP: {tp}, FP: {fp}, FN: {fn}, PRECISION: {precision:.4f}, RECALL: {recall:.4f}, F1 SCORE: {f1_score}')

true = train.loc[(train.video.isin(valid_videos))&(train.impact==1)&(train.confidence>1)&(train.visibility>0)]
print('There are %i ground truths for the videos in OOF file'%true.shape[0] )
print()
print('This OOF file has competition metric:')
competition_metric(true, df_pred)

# Make Videos
The code below can be cleaned up. The repeated code blocks can be put into a function

In [ ]:
# Modified function from to take single frame.
# https://www.kaggle.com/samhuddleston/nfl-1st-and-future-getting-started
def annotate_frame(gameKey, playID, video_labels, slow=1, stop_frame=-1, start_frame=-1) -> str:
    VIDEO_CODEC = "MP4V"
    BLACK = (0, 0, 0)    # Black
    WHITE = (255, 255, 255)    # White
    IMPACT_COLOR = (0, 0, 255)  # Red
    PRED_COLOR = (255, 0, 0) # Blue
    PRED_COLOR_WARN1 = (0, 255, 255) # Yellow
    PRED_COLOR_WARN2 = (0, 255, 0) # Green
    
    tp, fp, fn, pp, rr, ff = competition_metric(true.loc[(true.gameKey==gameKey)&(true.playID==playID)],
                               df_pred.loc[(df_pred.gameKey==gameKey)&(df_pred.playID==playID)],True)
    
    video_path1 = BASE_DIR+'/train/%i_%.6i_Endzone.mp4'%(gameKey,playID)
    video_path2 = BASE_DIR+'/train/%i_%.6i_Sideline.mp4'%(gameKey,playID)
    
    video_name1 = os.path.basename(video_path1)
    video_name2 = os.path.basename(video_path2)
    
    hits1 = train.loc[train.video==video_name1].drop_duplicates('frame').sort_values('frame').hit.values
    f_max1 = train.loc[train.video==video_name1,'frame'].max()
    hits2 = train.loc[train.video==video_name2].drop_duplicates('frame').sort_values('frame').hit.values
    f_max2 = train.loc[train.video==video_name2,'frame'].max()
    
    hits3 = df_pred.loc[df_pred.video==video_name1].frame.unique()
    hits4 = df_pred.loc[df_pred.video==video_name2].frame.unique()
    
    if f_max1 != f_max2:
        print('## WARNING: different length videos')
    f_max = min(f_max1,f_max2)
    print('Converting',f_max,'frames...',end='')
    
    vidcap1 = cv2.VideoCapture(video_path1)
    vidcap2 = cv2.VideoCapture(video_path2)
    
    fps = vidcap1.get(cv2.CAP_PROP_FPS)
    width1 = int(vidcap1.get(cv2.CAP_PROP_FRAME_WIDTH))
    height1 = int(vidcap1.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    output_path = "labeled_" + video_name1.replace('_Endzone','')
    tmp_output_path = "tmp_" + output_path
    output_video = cv2.VideoWriter(tmp_output_path, cv2.VideoWriter_fourcc(*VIDEO_CODEC), fps/slow, (width1, height1))
    
    frame = 0
    while True:
        
        if frame%10==0: print(frame,', ',end='')
        img = np.zeros((height1,width1,3),dtype='uint8')
                
        it_worked1, img1 = vidcap1.read()
        if not it_worked1: break
            
        it_worked2, img2 = vidcap2.read()
        if not it_worked2: break
            
        if frame<start_frame: 
            frame += 1
            continue
        if frame==stop_frame: break
            
        img[360:,:640,:] = img1[::2,::2,:]
        img[360:,640:,:] = img2[::2,::2,:]
        
        # We need to add 1 to the frame count to match the label frame index that starts at 1
        frame += 1
        
        # Let's add a frame index to the video so we can track where we are
        img_name = f"GamePlay_{video_name1.replace('_Endzone.mp4','')}_frame{frame}"
        cv2.putText(img, img_name, (0, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, WHITE, thickness=2)
        
        metric = f'TP: {tp}, FP: {fp}, FN: {fn}, PRECISION: {pp:.3f}, RECALL: {rr:.3f}, F1 SCORE: {ff:.4f}'
        cv2.putText(img, metric, (10, 300), cv2.FONT_HERSHEY_SIMPLEX, 0.5, WHITE, thickness=1)
            
        # MAKE FOUR PROGRESS LINES
        hh = 100
        cv2.line(img, (20,hh),(600,hh),(0,0,255),4)
        for k in np.where( hits1==1 )[0]:
            x = int(k/f_max * 580 + 20)
            cv2.rectangle(img, (x-1,hh-10),(x+1,hh+10),(0,0,255),cv2.FILLED) 
        x = int(frame/f_max * 580 + 20)
        cv2.rectangle(img, (x-1,hh-10),(x+1,hh+10),(255,255,255),cv2.FILLED) 
        
        hh = 150
        cv2.line(img, (20,hh),(600,hh),(0,0,255),4)
        for k in np.where( hits2==1 )[0]:
            x = int(k/f_max * 580 + 20)
            cv2.rectangle(img, (x-1,hh-10),(x+1,hh+10),(0,0,255),cv2.FILLED) 
        x = int(frame/f_max * 580 + 20)
        cv2.rectangle(img, (x-1,hh-10),(x+1,hh+10),(255,255,255),cv2.FILLED) 
        
        hh = 200
        cv2.line(img, (20,hh),(600,hh),(255,0,0),4)
        for k in hits3:
            x = int(k/f_max * 580 + 20)
            cv2.rectangle(img, (x-1,hh-10),(x+1,hh+10),(255,0,0),cv2.FILLED) 
        x = int(frame/f_max * 580 + 20)
        cv2.rectangle(img, (x-1,hh-10),(x+1,hh+10),(255,255,255),cv2.FILLED) 
        
        hh = 250
        cv2.line(img, (20,hh),(600,hh),(255,0,0),4)
        for k in hits4:
            x = int(k/f_max * 580 + 20)
            cv2.rectangle(img, (x-1,hh-10),(x+1,hh+10),(255,0,0),cv2.FILLED) 
        x = int(frame/f_max * 580 + 20)
        cv2.rectangle(img, (x-1,hh-10),(x+1,hh+10),(255,255,255),cv2.FILLED) 
        
        w1, w2, h1 = [], [], []
        
        # DRAW 4 SETS OF BOXES
        boxes = video_labels.query("video == @video_name1 and frame == @frame and warning != 0")
        for box in boxes.itertuples(index=False):
            left = box.left//2
            top = box.top//2 + 360
            width = box.width//2
            height = box.height//2
            if box.impact == 1 and box.confidence > 1 and box.visibility > 0:   
                color, thickness = IMPACT_COLOR, 2
                #print('(Impact frame',frame,box.label,box.confidence,box.visibility,')',end='')  
                h1.append(box.label)
            elif box.warning == 1:    
                color, thickness = WHITE, 1
                w1.append(box.label)
            else:
                color, thickness = BLACK, 1
                w2.append(box.label)
            # Add a box around the helmet
            cv2.rectangle(img, (left, top), (left + width, top + height), color, thickness=thickness)
            #cv2.putText(img, box.label, (left, max(0, top - 5//2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, thickness=1)
            
        # Now, add the boxes
        boxes = video_labels.query("video == @video_name2 and frame == @frame and warning != 0")
        for box in boxes.itertuples(index=False):
            left = box.left//2 + 640
            top = box.top//2 + 360
            width = box.width//2
            height = box.height//2
            if box.impact == 1 and box.confidence > 1 and box.visibility > 0:   
                color, thickness = IMPACT_COLOR, 2
                #print('Impact frame',frame,box.label,box.confidence,box.visibility)            
            elif box.warning == 1:    
                color, thickness = WHITE, 1
            else:
                color, thickness = BLACK, 1
            # Add a box around the helmet
            cv2.rectangle(img, (left, top), (left + width, top + height), color, thickness=thickness)
            #cv2.putText(img, box.label, (left, max(0, top - 5//2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, thickness=1)
            
            
        #Now, add the boxes
        boxes = df_pred.loc[(df_pred.video == video_name1) & (abs(df_pred.frame - frame)<=10)]
        for box in boxes.itertuples(index=False):
            left = box.left//2
            top = box.top//2 + 360
            width = box.width//2
            height = box.height//2
            if box.frame == frame:   
                color, thickness = PRED_COLOR, 2
                #print('(Pred frame',frame,')',end='')  
            elif box.frame > frame:
                color, thickness = PRED_COLOR_WARN1, 1
            else:
                color, thickness = PRED_COLOR_WARN2, 1
                
            # Add a box around the helmet
            cv2.rectangle(img, (left, top), (left + width, top + height), color, thickness=thickness)
            #cv2.putText(img, box.label, (left, max(0, top - 5//2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, thickness=1)
                              
        #Now, add the boxes
        boxes = df_pred.loc[(df_pred.video == video_name2) & (abs(df_pred.frame - frame)<=10)]
        for box in boxes.itertuples(index=False):
            left = box.left//2 + 640
            top = box.top//2 + 360
            width = box.width//2
            height = box.height//2
            if box.frame == frame:   
                color, thickness = PRED_COLOR, 2
                #print('(Pred frame',frame,')',end='') 
            elif box.frame > frame:
                color, thickness = PRED_COLOR_WARN1, 1
            else:
                color, thickness = PRED_COLOR_WARN2, 1

            # Add a box around the helmet
            cv2.rectangle(img, (left, top), (left + width, top + height), color, thickness=thickness)
            #cv2.putText(img, box.label, (left, max(0, top - 5//2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, thickness=1)
            
        # DRAW ARIAL VIEW WITH TRACKING INFO
        img3 = get_track_image(gameKey,playID,fps,frame+1,f_max, warn1=w1, warn2=w2, hit=h1)
        img[:360,640:,:] = img3[::2,::2,:]    
        
        
        output_video.write(img)
    output_video.release()
    
    # Not all browsers support the codec, we will re-load the file at tmp_output_path and convert to a codec that is more broadly readable using ffmpeg
    if os.path.exists(output_path):
        os.remove(output_path)
    subprocess.run(["ffmpeg", "-i", tmp_output_path, "-crf", "18", "-preset", "veryfast", "-vcodec", "libx264", output_path])
    os.remove(tmp_output_path)
    
    return output_path

In [ ]:
print('Here are some videos in OOF file')
valid_videos[:10]

In [ ]:
# CHOOSE GAME, PLAY, AND CREATE VIDEO
game = '58102_002798' 
g = int( game.split('_')[0] )
p = int( game.split('_')[1] )

# VARIABLE "SLOW" CONTROLS FRAME RATE
annotate_frame(g, p, video_labels=train, slow=15, start_frame = -1, stop_frame = -1 )

In [ ]:
display(Video(data='labeled_%i_%.6i.mp4'%(g,p), embed=True))